Example of dataloader usage. Unsure of how to load skin segmentation data at this time.

In [ ]:
# Datasest dir should look like:


# ├── data
# │   ├── v1_cam1_no_split
# |      ├── Adjust Radio
# |          ├── 1.jpg
# │      ├── Drink
# |           ├── 1.jpg
# |      ├── Test_data_list.csv
# |      ├── Train_data_list.csv
# |
# │   ├── v2_cam1_cam2_ split_by_driver
# |      ├── Camera 1
# |          ├── test
# |          ├── train
# │      ├── Camera 2
# |          ├── test
# |          ├── train
# |      ├── skin_nonskin_pixels


In [ ]:
from dataloaders import *
import torchvision.transforms as T


transform = T.Compose([
    T.ToTensor(),
    T.Resize((224, 224)),
])

v1_train = V1Dataset(csv_path='data/v1_cam1_no_split/Train_data_list.csv', images_folder='data/v1_cam1_no_split', 
                     transform=transform)
v1_test = V1Dataset(csv_path='data/v1_cam1_no_split/Test_data_list.csv', images_folder='data/v1_cam1_no_split', 
                    transform=transform)

v2_train = V2Dataset(cam1_path='data/v2_cam1_cam2_ split_by_driver/Camera 1/train', 
                     cam2_path='data/v2_cam1_cam2_ split_by_driver/Camera 2/train', transform=transform)
v2_test = V2Dataset(cam1_path='data/v2_cam1_cam2_ split_by_driver/Camera 1/test', 
                    cam2_path='data/v2_cam1_cam2_ split_by_driver/Camera 2/test', transform=transform)

v1_trainloader = DataLoader(v1_train, batch_size=32, shuffle=True)
v1_testloader = DataLoader(v1_test, batch_size=32, shuffle=True)

v2_trainloader = DataLoader(v2_train, batch_size=32, shuffle=True)
v2_testloader = DataLoader(v2_test, batch_size=32, shuffle=True)

In [ ]:
print(f'Len of v1_trainloader: {len(v1_trainloader)}')
for images, label in v1_trainloader:
    print(images.shape)
    print(label.shape)
    break


In [ ]:
print(f'Len of v1_trainloader: {len(v1_testloader)}')
for images, label in v1_testloader:
    print(images.shape)
    print(label.shape)
    break

In [ ]:
print(f'Len of v2_trainloader: {len(v2_trainloader)}')
for images, label in v2_trainloader:
    print(images.shape)
    print(label.shape)
    break

In [ ]:
print(f'Len of v1_trainloader: {len(v2_testloader)}')
for images, label in v2_testloader:
    print(images.shape)
    print(label.shape)
    break

In [ ]:
from torchvision.models import squeezenet1_1, SqueezeNet1_1_Weights

from torchvision.models._api import WeightsEnum
from torch.hub import load_state_dict_from_url
import torch.nn as nn
import torch

def get_state_dict(self, *args, **kwargs):
    kwargs.pop("check_hash")
    return load_state_dict_from_url(self.url, *args, **kwargs)

WeightsEnum.get_state_dict = get_state_dict

model = squeezenet1_1(weights=SqueezeNet1_1_Weights.DEFAULT)
model.classifier[1].out_channels = 10

model.classifier

# input = torch.randn(64, 3, 227, 227)

# output = nn.Flatten()(feature_extractor(input))

# output.shape

In [ ]:
len(list(model.features.parameters()))

In [ ]:
class CustomWeightsEnum(WeightsEnum):
    def get_state_dict(self, *args, **kwargs):
            kwargs.pop("check_hash")
            return load_state_dict_from_url(self.url, *args, **kwargs)



In [ ]:
from torchvision.models import inception_v3, Inception_V3_Weights


inception_model = inception_v3(weights=Inception_V3_Weights.DEFAULT)

inception_model.fc.out_features = 10 
inception_model.fc

In [1]:
from cnn.hands_cnn import Hands_VGG16
from wrappers import *
import argparse


args = argparse.ArgumentParser()
args.add_argument('--model', type=str, default='hands_vgg')
args.add_argument('--distributed', type=bool, default=False)

args.add_argument('--resume_path', type=str, default=None)
args.add_argument('--resume_last_epoch', type=bool, default=False)

args.add_argument('--hidden_units', type=int, default=128)
args.add_argument('--freeze', type=bool, default=True)
args.add_argument('--batch_size', type=int, default=64)
args.add_argument('--epochs', type=int, default=30)

args.add_argument('--lr', type=float, default=1e-3)
args.add_argument('--num_frozen_params', type=int, default=30)
args.add_argument('--dropout', type=float, default=0.5)
args.add_argument('--optimizer', type=str, default='SGD')
args.add_argument('--weight_decay', type=float, default=0.0)
args.add_argument('--scheduler', action='store_true')

args.add_argument('--save_period', type=int, default=5)
args.add_argument('--transform', type=bool, default=True) 
args.add_argument('--device', type=str, default='cuda')

args.add_argument('--data_dir', type=str, default='data/v2_cam1_cam2_split_by_driver')
args.add_argument('--model_dir', type=str, default='cnn/hands_models')
args.add_argument('--detector_path', type=str, default='path/to/yolo/weights')

args = args.parse_args()

model = Hands_VGG16(args, 10)

model.load_state_dict(torch.load("/home/ron/Classes/CV-Systems/cybertruck/cnn/hands_models/vgg/epoch60_11-16_03:44:44.pt"))

wrapper_hands = Hands_Inference_Wrapper(model).to(args.device)

wrapper_hands


Hands_Inference_Wrapper(
  (resize): Resize(size=[640, 640], interpolation=InterpolationMode.BILINEAR, antialias=warn)
  (detector): YOLO(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (2): C2f(
          (cv1): Conv(
            (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (cv2): C

In [ ]:
wrapper_hands.detector.val()

In [2]:
import torch.onnx


x = torch.randn(64, 3, 224, 224).to('cuda')

torch_out = wrapper_hands(x)


/home/ron/miniconda3/envs/cyber/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
WARNING ⚠️ torch.Tensor inputs should be normalized 0.0-1.0 but max value is 4.718448638916016. Dividing input by 255.


In [3]:
torch.onnx.export(wrapper_hands, x, 'ensemble.onnx', export_params=True, opset_version=10, do_constant_folding=True, input_names = ['input'], 
                  output_names = ['output'], dynamic_axes={'input' : {0 : 'batch_size'}, 'output' : {0 : 'batch_size'}})

New https://pypi.org/project/ultralytics/8.0.211 available 😃 Update with 'pip install -U ultralytics'
engine/trainer: task=detect, mode=train, model=detection/hands_detection/runs/detect/best/weights/best.pt, data=coco8.yaml, epochs=100, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False,

RuntimeError: GET was unable to find an engine to execute this computation